In [ ]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [ ]:
import minsearch
import json
import requests 

In [ ]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
documents[0]

In [ ]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section']
    , keyword_fields=['course']
)

In [ ]:
index.fit(documents)

In [ ]:
from openai import OpenAI
client = OpenAI()

In [17]:
def search(query):
    boost = {
        'question': 3.0
    }
    
    results = index.search(
        query=query
        , boost_dict=boost
        , filter_dict={ 'course': 'data-engineering-zoomcamp' }
        , num_results=5
    )
    

In [18]:
def build_prompt(query, search_results):
    prompt_template = """
    You are a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """
    
    context = ""
    
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}"
    
    prompt=prompt_template.format(question=query, context=context).strip()

    return prompt

In [20]:
def llm(prompt):
    response = client.chat.completitions.create(
        model="gpt-4o"
        , messages=[{"role": "user", "content": prompt}]
    )

    return reponse.choices[0].message.content
    

In [21]:
query = "how do I set up docker"

def rag(query):
    results = search(query)
    prompt = build_prompt(query, results)
    answer = llm(prompt)

    return answer

In [ ]:
rag(query)